In [ ]:
#to add package, go to REPL
#using Pkg
#Pkg.add("Package Name")
using DataFrames
using RDatasets
using Statistics
using StatsPlots
using CSV
using StatsBase
using NamedArrays
using PrettyTables

using LIBSVM
using Printf
# using SVR
using MLJ
using Mads
using Metrics
using ScikitLearn.GridSearch: GridSearchCV

using PyCall

In [ ]:
skSVR = pyimport("sklearn.svm")

In [ ]:
df = CSV.read(joinpath(dirname(pwd()), "Concrete_Data.csv"), DataFrame)

In [ ]:
#splitting into 70% training, 15% test, 15% validation

dfm = Matrix(df)
# dfms = dfm[shuffle(1:end), :]
dfms = dfm

x = dfms[:, 1:8]
y = dfms[:, 9]

data_split = [0.7, 0.15, 0.15]
split_cs2 = Int.(round.(cumsum(data_split)*length(y)))
split_cs1 = vcat(1, split_cs2[1:(end-1)].+1)

xtrain = x[split_cs1[1]:split_cs2[1], :]
ytrain = y[split_cs1[1]:split_cs2[1]]

xtest = x[split_cs1[2]:split_cs2[2], :]
ytest = y[split_cs1[2]:split_cs2[2]]

xval = x[split_cs1[3]:split_cs2[3], :]
yval = y[split_cs1[3]:split_cs2[3]];

In [ ]:
#training the model, setting the hypermeters C, degree, gamma. Want to use gridsearch to optimise hyperparameters. Not sure what SVR.train does #how to do gradient descent for hyperparameters

# pmodel = SVR.train(ytrain, permutedims(xtrain),; kernel_type=SVR.RBF, C= 10.0, degree = 1 , gamma = 0.00003)

# y_pr = SVR.predict(pmodel, permutedims(xtest))
# mse_SVR = mse(y_pr, ytest)

In [ ]:
param_grid = Dict(:C => 100:100:1000,
                :gamma => :0.0000001:0000001:0.000001,
                :degree => 0.1:0.1:1)

grid = GridSearchCV(skSVR.SVR(), param_grid, refit = true, verbose = 3)
ScikitLearn.fit!(grid ,xtrain, ytrain)

In [ ]:
println("Best hyper-parameters: $(grid.best_params_)")

In [ ]:
regressor = ScikitLearn.fit!(skSVR.SVR(C=1000.0, gamma=1.0e-7, degree =0.1),xtrain,ytrain)
y_pr = ScikitLearn.predict(regressor,xtest)
mse_SVR = mse(y_pr, ytest)

# C=50.0, gamma=0.000003, degree =1

In [ ]:
#plotting graph to see predicted vs true
compare_plt = plot(ytest, y_pr, 
st=:scatter,
xlims=(0,80),
ylims=(0,80),
xlab = "True Value (ytest)",
ylab = "Predicted Value (y_pr)",
size = (500,500))

xx = 0:80
yy = 0:80

plot!(compare_plt, xx, yy, st=:line)

mse_SVR = mse(y_pr, ytest)
annotate!(15,75, text("mse = $(round(mse_SVR, sigdigits=2))", 10))

display(compare_plt)